In [1]:
import json
from bqm_input_checker import InputChecker
from btor2bqm import BTor2BQM

The code we execute: 

```C
uint64_t* x;
uint64_t main() { 
    uint64_t a;
    x = malloc(1); // rounded up to 8
    
    // touch to trigger page fault here
    *x = 0;
    // read 1 byte from console into x
    read(0, x, 1);
    a = *x;
    a = *(x + a); // segfault if input != 0
}
```

# 32 bit example (u3.btor2)

```` bash
./modeler-32 -c u3.c - 0 --no-syscall-id-check --no-exit-code-check --no-division-by-zero-check --no-address-alignment-check --constant-propagation --linear-address-space
````

In [2]:
parser = BTor2BQM(15)
bqm = parser.parse_file(f"../dummy_btor2files/u3.btor2",
                  f"../all_bad_small_sample/",
                  with_init=True, modify_memory_sort=True)

started building ../btor2files/u3.btor2 for 15 timesteps
{'size_datasegment': 2, 'size_heap': 1, 'size_stack': 10, 'begin_datasegment': 17408, 'begin_heap': 18432, 'word_size': 32, 'address_step_size': 1, 'address_word_size': 30, 'begin_stack': 1073741824}
output dir:  ../all_bad_small_sample/
sort memory modified to be bitvector of size:  416


In [3]:
len(bqm.linear.keys())

351

In [4]:
# check that the model we generated is good.
for i in range(0, 256):
    bias, error_states = InputChecker.run_checker(f"../all_bad_small_sample/", i)
    print(f"{i}: ",bias, error_states)

0:  2.0 []
1:  0.0 [80000009]
2:  0.0 [80000009]
3:  0.0 [80000009]
4:  0.0 [80000009]
5:  0.0 [80000009]
6:  0.0 [80000009]
7:  0.0 [80000009]
8:  0.0 [80000009]
9:  0.0 [80000009]
10:  0.0 [80000009]
11:  0.0 [80000009]
12:  0.0 [80000009]
13:  0.0 [80000009]
14:  0.0 [80000009]
15:  0.0 [80000009]
16:  0.0 [80000009]
17:  0.0 [80000009]
18:  0.0 [80000009]
19:  0.0 [80000009]
20:  0.0 [80000009]
21:  0.0 [80000009]
22:  0.0 [80000009]
23:  0.0 [80000009]
24:  0.0 [80000009]
25:  0.0 [80000009]
26:  0.0 [80000009]
27:  0.0 [80000009]
28:  0.0 [80000009]
29:  0.0 [80000009]
30:  0.0 [80000009]
31:  0.0 [80000009]
32:  0.0 [80000009]
33:  0.0 [80000009]
34:  0.0 [80000009]
35:  0.0 [80000009]
36:  0.0 [80000009]
37:  0.0 [80000009]
38:  0.0 [80000009]
39:  0.0 [80000009]
40:  0.0 [80000009]
41:  0.0 [80000009]
42:  0.0 [80000009]
43:  0.0 [80000009]
44:  0.0 [80000009]
45:  0.0 [80000009]
46:  0.0 [80000009]
47:  0.0 [80000009]
48:  0.0 [80000009]
49:  0.0 [80000009]
50:  0.0 [80000009

In [5]:
from dwave.system import DWaveSampler, EmbeddingComposite
from greedy import SteepestDescentComposite

qpu = EmbeddingComposite(DWaveSampler(solver={"name": "Advantage_system4.1"}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm, num_reads=7000, chain_strength=1.5)

In [6]:
result.first.energy

0.0

In [12]:
with open(f"../all_bad_small_sample/context.json") as json_file:
    context = json.load(json_file)

In [14]:
input_qubit_names = context['input']
res = ""
for name in input_qubit_names:
    res += str(result.first.sample[name])
print(res[::-1])

11110101


In [22]:
for low in result.lowest():
    res = ""
    for name in input_qubit_names:
        res += str(low[name])
    print(res[::-1])

11110101
11010111
11010100
11000100
11110110
11100000
10100010
11000000
11110000


# 64 bit small sample

````bash
./modeler -c u3.c - 0 --no-syscall-id-check --no-exit-code-check --no-division-by-zero-check --no-address-alignment-check --constant-propagation --linear-address-space
````

In [23]:
parser = BTor2BQM(15)
bqm = parser.parse_file(f"../btor2files/u3-64.btor2",
                  f"../64bit_small_sample/",
                  with_init=True, modify_memory_sort=True)

started building ../btor2files/u3-64.btor2 for 15 timesteps
{'size_datasegment': 2, 'size_heap': 1, 'size_stack': 9, 'begin_datasegment': 8704, 'begin_heap': 9216, 'word_size': 64, 'address_step_size': 1, 'address_word_size': 29, 'begin_stack': 536870912}
output dir:  ../64bit_small_sample/
sort memory modified to be bitvector of size:  768


In [24]:
len(bqm.linear.keys())

404

In [26]:
from dwave.system import DWaveSampler, EmbeddingComposite
from greedy import SteepestDescentComposite

qpu = EmbeddingComposite(DWaveSampler(solver={"name": "Advantage_system4.1"}))
sampler = SteepestDescentComposite(qpu)
result = sampler.sample(bqm, num_reads=5000, chain_strength=1.5)

In [27]:
result.first.energy

0.0

In [28]:
with open(f"../64bit_small_sample/context.json") as json_file:
    context = json.load(json_file)
input_qubit_names = context['input']
for low in result.lowest():
    res = ""
    for name in input_qubit_names:
        res += str(low[name])
    print(res[::-1])

00111011


In [30]:
# Without steepest gradient descent
from dwave.system import DWaveSampler, EmbeddingComposite
from greedy import SteepestDescentComposite

qpu = EmbeddingComposite(DWaveSampler(solver={"name": "Advantage_system4.1"}))
# sampler = SteepestDescentComposite(qpu)
result = qpu.sample(bqm, num_reads=7000, chain_strength=1.5)
result.first.energy

0.0

In [31]:
with open(f"../64bit_small_sample/context.json") as json_file:
    context = json.load(json_file)
input_qubit_names = context['input']
for low in result.lowest():
    res = ""
    for name in input_qubit_names:
        res += str(low[name])
    print(res[::-1])

01010011
